<a href="https://colab.research.google.com/github/HoTrungQuan2004/ParkingAreaManage/blob/main/License_plate_detection_tflite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install tensorflow
!pip install tensorflow.keras

In [11]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from glob import glob

In [12]:
from google import colab
colab.drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
#load images and labels
def load_data(img_dir, label_dir):
    X, y = [], []
    for img_path in glob(f"{img_dir}/*.jpg"):
        img = cv2.imread(img_path)
        img = cv2.resize(img, (96, 96))
        label_path = os.path.join(label_dir, os.path.basename(img_path).replace('.jpg', '.txt'))
        with open(label_path) as f:
            line = f.readline().strip().split()
            _, x, y_, w, h = map(int, line)
            x, y_, w, h = x / 96, y_ / 96, w / 96, h / 96
            X.append(img / 255.0)
            y.append([x, y_, w, h])
    return np.array(X), np.array(y)

X, y = load_data("images", "labels")  #path

#CNN model
model = models.Sequential([
    layers.Input(shape=(96,96,3)),
    layers.Conv2D(16, 3, activation='relu'), layers.MaxPooling2D(),
    layers.Conv2D(32, 3, activation='relu'), layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu'), layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(4, activation='sigmoid')  # x, y, w, h
])
model.compile(optimizer='adam', loss='mse')
model.fit(X, y, epochs=20, batch_size=16)

In [ ]:
#convert into .tflite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
with open("plate_detector.tflite", "wb") as f:
    f.write(tflite_model)

In [ ]:
!xxd -i plate_detector.tflite > plate_model.h